In [2]:
from fmm import Network,NetworkGraph,FastMapMatch,FastMapMatchConfig,UBODT,GPSConfig,ResultConfig
import pandas as pd
from shapely.wkt import loads
from shapely.geometry import mapping
import json
from ipyleaflet import Map, GeoJSON, WidgetControl
import ipywidgets as widgets

In [3]:
#load network data and graph 
network = Network("../osmnx_example/beijing/edges.shp","fid", "u", "v")
print("Nodes {} edges {}".format(network.get_node_count(),network.get_edge_count()))
graph = NetworkGraph(network)

[2025-02-24 17:43:00.901] [info] [network.cpp:72] Read network from file ../osmnx_example/beijing/edges.shp
[2025-02-24 17:43:01.356] [info] [network.cpp:172] Number of edges 85538 nodes 33999
[2025-02-24 17:43:01.356] [info] [network.cpp:173] Field index: id 18 source 0 target 1
[2025-02-24 17:43:01.392] [info] [network.cpp:176] Read network done.
Nodes 33999 edges 85538
[2025-02-24 17:43:01.393] [info] [network_graph.cpp:17] Construct graph from network edges start
[2025-02-24 17:43:01.400] [info] [network_graph.cpp:30] Graph nodes 33999 edges 85538
[2025-02-24 17:43:01.400] [info] [network_graph.cpp:31] Construct graph from network edges end


### Precompute an UBODT file

**This step can be skipped if you already created one UBODT file.**

In [3]:
#from fmm import UBODTGenAlgorithm
#ubodt_gen = UBODTGenAlgorithm(network,graph)
#status = ubodt_gen.generate_ubodt("../osmnx_example/beijing/ubodt.txt", 4, binary=False, use_omp=True)
#print(status)

### FMM model creation and map matching

match single trajectory

In [4]:
df = pd.read_csv("../osmnx_example/beijing/trajectories.csv", sep=";")

#trajectory 1
wkt = df.loc[df["id"] == 71, "geom"].values[0]

print(wkt)

ubodt = UBODT.read_ubodt_csv("../osmnx_example/beijing/ubodt.txt")
model = FastMapMatch(network,graph,ubodt)

# configuration parameters
k = 5
radius = 0.4
gps_error = 0.5
fmm_config = FastMapMatchConfig(k,radius,gps_error)

result = model.match_wkt(wkt,fmm_config)
print("Matched path: ", list(result.cpath))
print("Matched edge for each point: ", list(result.opath))
print("Matched edge index ",list(result.indices))
print("Matched geometry: ",result.mgeom.export_wkt())
print("Matched point ", result.pgeom.export_wkt())

LINESTRING(116.30653 40.013794, 116.306538 40.013833, 116.306339 40.013967, 116.306288 40.014051, 116.30616 40.014113, 116.306025 40.014114, 116.30589 40.014113, 116.305761 40.014096, 116.305688 40.014017, 116.305682 40.013926, 116.305693 40.013823, 116.305761 40.013747, 116.305793 40.01374, 116.305798 40.013738, 116.305803 40.013737, 116.305805 40.013735, 116.305805 40.013735, 116.305812 40.013733, 116.30582 40.013721, 116.30593 40.013736, 116.306069 40.013753, 116.306208 40.013759, 116.306331 40.013719, 116.306378 40.01363, 116.306465 40.013538, 116.306563 40.013507, 116.306594 40.013498, 116.306672 40.013459, 116.306753 40.01339, 116.306841 40.013308, 116.306943 40.01323, 116.307055 40.013167, 116.307148 40.013094, 116.307246 40.013034, 116.307385 40.013004, 116.307504 40.013018, 116.307649 40.013022, 116.307795 40.013029, 116.307928 40.013047, 116.308066 40.013046, 116.308207 40.013053, 116.308325 40.013062, 116.308457 40.013083, 116.308598 40.01309, 116.308746 40.013101, 116.30889

In [5]:
import folium
from shapely.wkt import loads

original_geom = loads(wkt)
matched_geom = loads(result.mgeom.export_wkt())

m = folium.Map(location=[original_geom.coords[0][1], original_geom.coords[0][0]], zoom_start=16)

folium.PolyLine([(lat, lon) for lon, lat in original_geom.coords], color="red", weight=3, opacity=0.8, tooltip="Original GPS Trajectory").add_to(m)
folium.PolyLine([(lat, lon) for lon, lat in matched_geom.coords], color="blue", weight=3, opacity=0.8, tooltip="Matched Path").add_to(m)

for lon, lat in original_geom.coords:
    folium.CircleMarker(location=[lat, lon], radius=3, color="red", fill=True, fill_color="red").add_to(m)
for lon, lat in matched_geom.coords:
    folium.CircleMarker(location=[lat, lon], radius=3, color="blue", fill=True, fill_color="blue").add_to(m)

# Display map 
m

### Map Matching after obfuscation

In [6]:
df = pd.read_csv("../osmnx_example/beijing/perturbed_trajectories.csv", sep=";")

#trajectory 1
wkt_pert = df.loc[df["id"] == 71, "geom"].values[0]

print(wkt_pert)

# configuration parameters
k = 5
radius = 0.4
gps_error = 0.5
fmm_config = FastMapMatchConfig(k,radius,gps_error)

result_pert = model.match_wkt(wkt_pert,fmm_config)
print("Matched path: ", list(result_pert.cpath))
print("Matched edge for each point: ", list(result_pert.opath))
print("Matched edge index ",list(result_pert.indices))
print("Matched geometry: ",result_pert.mgeom.export_wkt())
print("Matched point ", result_pert.pgeom.export_wkt())

LINESTRING(116.3083765778394 40.012251240137985, 116.3083765778394 40.012251240137985, 116.3083765778394 40.012251240137985, 116.3083765778394 40.012251240137985, 116.3083765778394 40.012251240137985, 116.3083765778394 40.012251240137985, 116.3083765778394 40.012251240137985, 116.30727889074531 40.01552649485553, 116.30727889074531 40.01552649485553, 116.30727889074531 40.01552649485553, 116.30727889074531 40.01552649485553, 116.30727889074531 40.01552649485553, 116.30727889074531 40.01552649485553, 116.30727889074531 40.01552649485553, 116.30727889074531 40.01552649485553, 116.30727889074531 40.01552649485553, 116.30727889074531 40.01552649485553, 116.30727889074531 40.01552649485553, 116.30727889074531 40.01552649485553, 116.30727889074531 40.01552649485553, 116.30727889074531 40.01552649485553, 116.30727889074531 40.01552649485553, 116.30727889074531 40.01552649485553, 116.30727889074531 40.01552649485553, 116.3083765778394 40.012251240137985, 116.3083765778394 40.012251240137985, 1

In [ ]:
import folium
from shapely.wkt import loads

original_geom = loads(wkt)
pert_geom = loads(wkt_pert)
pert_matched_geom = loads(result_pert.mgeom.export_wkt())

m = folium.Map(location=[original_geom.coords[0][1], original_geom.coords[0][0]], zoom_start=16)

folium.PolyLine([(lat, lon) for lon, lat in original_geom.coords], color="red", weight=3, opacity=0.8, tooltip="Original GPS Trajectory").add_to(m)
folium.PolyLine([(lat, lon) for lon, lat in pert_geom.coords], color="blue", weight=3, opacity=0.8, tooltip="Perturbed GPS Trajectory").add_to(m)
folium.PolyLine([(lat, lon) for lon, lat in pert_matched_geom.coords], color="green", weight=3, opacity=0.8, tooltip="Matched Path").add_to(m)

for lon, lat in original_geom.coords:
    folium.CircleMarker(location=[lat, lon], radius=3, color="red", fill=True, fill_color="red").add_to(m)
for lon, lat in pert_geom.coords:
    folium.CircleMarker(location=[lat, lon], radius=3, color="blue", fill=True, fill_color="blue").add_to(m)
for lon, lat in pert_matched_geom.coords:
    folium.CircleMarker(location=[lat, lon], radius=3, color="green", fill=True, fill_color="green").add_to(m)
# Display map 
m